In [1]:
# Import Dependencies
import psycopg2
import csv
from config import mypass

#### Connect to Database

In [2]:
# Connect to Database 
con = psycopg2.connect(user="postgres", password=mypass, host="127.0.0.1", port="5432")

print("Opened successfully")

Opened successfully


In [3]:
con.autocommit = True

cur = con.cursor()

#### Create tables and load data

In [4]:
# Create new flight database (drop old flights database if exists)

try:
    cur.execute('''DROP DATABASE flights;''')
    print ('Database dropped')   
except:
    print ('Database does not exist')
con.commit()


try:
    cur.execute('''CREATE DATABASE flights;''')
    print("Database created")   
except:
    print ('Database already exists')

con.commit()


Database dropped
Database created


In [5]:
# Close connection to database
con.close()

In [6]:
# Connect to flight Database 
con = psycopg2.connect(database="flights", user="postgres", password=mypass, host="127.0.0.1", port="5432")

print("Flights database opened successfully")

Flights database opened successfully


In [7]:
cur = con.cursor()

In [8]:
# Create table flights

cur.execute('''CREATE TABLE flights (
    "Unnamed: 0" BIGINT,
    month SMALLINT,
    day SMALLINT,
    day_of_week SMALLINT,
    airline VARCHAR(2),
    flight_number SMALLINT,
    tail_number VARCHAR(6),
    origin_airport VARCHAR(5),
    destination_airport VARCHAR(5),
    scheduled_departure SMALLINT,
    departure_delay DECIMAL,
    elapsed_time DECIMAL,
    air_time DECIMAL,
    distance SMALLINT,
    wheels_on DECIMAL,
    taxi_in DECIMAL,
    scheduled_arrival SMALLINT,
    arrival_time DECIMAL,
    arrival_delay DECIMAL,
    air_system_delay DECIMAL,
    security_delay DECIMAL,
    airline_delay DECIMAL,
    late_aircraft_delay DECIMAL,
    weather_delay DECIMAL,
    PRIMARY KEY ("Unnamed: 0")
    );''')

con.commit()
print("Flights table created successfully")

Flights table created successfully


In [9]:
# Load csv into flights table

with open('../resources/flights_cleaned.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'flights', sep=',', null='')    

con.commit()
print("Data loaded successfully")

Data loaded successfully


In [10]:
# Create table airlines
cur.execute('''CREATE TABLE airlines (
    iata_code VARCHAR(2),
    airline VARCHAR(30),
    PRIMARY KEY (iata_code)
);''')

con.commit()
print("Airlines table created successfully")

Airlines table created successfully


In [11]:
# Load csv into airlines table

with open('../resources/airlines.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'airlines', sep=',', null='')
    
con.commit()
print("Data loaded successfully")

Data loaded successfully


In [12]:
# Create table airports
cur.execute('''CREATE TABLE airports (
    iata_code VARCHAR(3),
    airport VARCHAR(80),
    city VARCHAR(35),
    state VARCHAR(2),
    country VARCHAR(3),
    latitude DECIMAL,
    longitude DECIMAL,
    PRIMARY KEY (iata_code)
);''')

con.commit()
print("Airports table created successfully")

Airports table created successfully


In [13]:
# Load csv into airports table

with open('../resources/airports.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'airports', sep=',', null="")
    
con.commit()
print("Data loaded successfully")

Data loaded successfully


#### Join tables

In [14]:
cur.execute('''SELECT f."Unnamed: 0",
    f.month,
    f.day,
    f.day_of_week,
    f.airline,
    f.flight_number,
    f.tail_number,
    f.origin_airport,
    a.state as origin_state,
    f.destination_airport,
    ai.state as destination_state,
    f.distance,
    f.departure_delay,
    f.arrival_delay
INTO flights_state
FROM flights as f
    LEFT JOIN airports as a
    ON (f.origin_airport = a.iata_code)
    LEFT JOIN airports as ai
    ON (f.destination_airport = ai.iata_code)
;''')

con.commit()
print('Tables joined successfully')

Tables joined successfully


In [15]:
cur.execute('''SELECT f."Unnamed: 0" as flight_id,
    f.month,
    f.day,
    f.day_of_week,
    f.airline,
    al.airline as airline_name,
    f.flight_number,
    f.tail_number,
    f.origin_airport,
    ar.airport as origin_airport_name,
    f.destination_airport,
    arp.airport as destination_airport_name,
    f.scheduled_departure,
    f.departure_delay,
    f.elapsed_time,
    f.air_time,
    f.distance,
    f.wheels_on,
    f.taxi_in,
    f.scheduled_arrival,
    f.arrival_time,
    f.arrival_delay,
    f.air_system_delay,
    f.security_delay,
    f.airline_delay,
    f.late_aircraft_delay,
    f.weather_delay
INTO flights_complete
FROM flights as f
    LEFT JOIN airlines as al
    ON (f.airline = al.iata_code)
    LEFT JOIN airports as ar
    ON (f.origin_airport = ar.iata_code)
    LEFT JOIN airports as arp
    ON (f.destination_airport = arp.iata_code);''')

con.commit()
print('Tables joined successfully')

Tables joined successfully


#### Create another table with a random sample of 10,000 rows, for the dashboard

In [17]:
# cur.execute('''
#     CREATE TABLE flights_complete_sample AS
#     (SELECT * FROM flights_complete 
#     ORDER BY RANDOM() 
#     LIMIT 9999);
# ''')
# con.commit()
# print('Table created successfully')

#### Close connection to database

In [18]:
# Close connection to database
con.close()
print("Closed successfully")

Closed successfully
